In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
import os
from dotenv import load_dotenv
import pathlib
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import dsp
import numpy as np
from scipy import sparse
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import faiss
import json
from dsp.utils.utils import deduplicate

lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8090, url="http://127.0.0.1")
dspy.settings.configure(lm=lm)


def create_faiss_index(df, text_column, id_column, model_name="all-mpnet-base-v2", index_file="faiss_index.index"):
    """
    Create a FAISS index from a DataFrame containing text data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    text_column (str): The name of the column containing text data.
    id_column (str): The name of the column containing unique identifiers for the texts.
    model_name (str): The name of the SentenceTransformer model to use for embeddings.
    index_file (str): The file path to save the FAISS index.

    Returns:
    index: The FAISS index object.
    model: The SentenceTransformer model used for embeddings.
    ids: List of document IDs.
    texts: List of document texts.
    """
    texts = df[text_column].tolist()
    ids = df[id_column].tolist()

    model = SentenceTransformer(model_name, device="cuda")

    # Calculate embeddings for the texts
    embeddings = model.encode(texts, show_progress_bar=False)

    # Create a FAISS index
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  

    # Normalize embeddings to unit length and add to index
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # Save the index to a file
    faiss.write_index(index, index_file)

    return index, model, ids, texts

def retrieve_similar_documents(query_text, model, index, ids, texts, k=5):
    """
    Retrieve the k most similar documents to the query text.

    Parameters:
    query_text (str): The query text.
    model: The SentenceTransformer model used for embeddings.
    index: The FAISS index object.
    ids (list): List of document IDs.
    texts (list): List of document texts.
    k (int): The number of nearest neighbors to retrieve.

    Returns:
    list: A list of dictionaries containing document IDs, distances, and texts of the k most similar documents.
    """
    # Encode the query text
    query_embedding = model.encode([query_text], show_progress_bar=False)
    faiss.normalize_L2(query_embedding)
    
    # Search the index for the k nearest neighbors
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve the corresponding texts and ids
    results = []
    for i in range(k):
        result = {
            "document_id": ids[indices[0][i]],
            "distance": distances[0][i],
            "text": texts[indices[0][i]]
        }
        results.append(result)
    
    return results


############
# DATA #####
############
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/translated_stops_filtered_by_al/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/LDA_FILTERED_AL/rosie_1_20")
path_corpus_en = path_model / "train_data" / "corpus_EN.txt"
path_corpus_es = path_model / "train_data" / "corpus_ES.txt"

persist_directory = (path_model / 'db_contr_mono').as_posix()

raw = pd.read_parquet(path_source)
with path_corpus_en.open("r", encoding="utf-8") as f:
    lines = [line for line in f.readlines()]
corpus_en = [line.rsplit(" 0 ")[1].strip().split() for line in lines]

ids = [line.split(" 0 ")[0] for line in lines]
df_en = pd.DataFrame({"lemmas": [" ".join(doc) for doc in corpus_en]})
df_en["doc_id"] = ids
df_en["len"] = df_en['lemmas'].apply(lambda x: len(x.split()))
df_en["id_top"] = range(len(df_en))
df_en_raw = df_en.merge(raw, how="inner", on="doc_id")[["doc_id", "id_top", "id_preproc", "lemmas_x", "text", "len"]]

# Read thetas 
thetas = sparse.load_npz(path_model.joinpath(f"mallet_output/{'EN'}/thetas.npz")).toarray()
betas = np.load((path_model.joinpath(f"mallet_output/{'EN'}/betas.npy")))
def get_thetas_str(row,thetas):
    return " ".join([f"{id_}|{round(el, 4)}" for id_,el in enumerate(thetas[row]) if el!=0.0])

def get_most_repr_tpc(row,thetas):
    return np.argmax(thetas[row])

# Save thetas in dataframe and "assigned topic"
df_en_raw["thetas"] = df_en_raw.apply(lambda row: get_thetas_str(row['id_top'], thetas), axis=1)
df_en_raw["id_tpc"] = df_en_raw.apply(lambda row: get_most_repr_tpc(row['id_top'], thetas), axis=1)
tpc = 1
df_tpc = df_en_raw[df_en_raw.id_tpc == tpc]

In [249]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [48]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

In [111]:
import ast

In [49]:
llm = dspy.OpenAI(
    model="gpt-3.5-turbo" , #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000)

# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm, trace=[])

In [68]:
lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8090, url="http://127.0.0.1")
dspy.settings.configure(lm=lm)

In [198]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

In [2]:
class GenerateFacts(dspy.Signature):
    """Extract the claims from the given text."""
    text = dspy.InputField(desc="may contain one or several claims")
    claims = dspy.OutputField(desc="List of sentences in the form 'subject + verb + object'")
    
class GenerateQuestion(dspy.Signature):
    """Form a close-ended question that directly asks the fact."""
    fact = dspy.InputField()
    question = dspy.OutputField(desc="it asks the fact")

class QAGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.ChainOfThought(GenerateFacts)#"text -> claims"
        self.generate_question = dspy.ChainOfThought(GenerateQuestion)
        
    def facts_check(self, text):
        correct = False
        try:
            parsed_facts = ast.literal_eval(text)
            if len(parsed_facts[0]) <=1:
                correct = False
            else:
                correct = True
        except Exception as e:
            print(f"-- -- Error: {e}")
            try:
                pattern = r'\d+\.\s'
                parsed_facts = re.split(pattern, text)
                parsed_facts = [s.strip() for s in parsed_facts if s]
                if len(parsed_facts[0]) == 0:
                    correct = False
                else:
                    correct = True
            except Exception as e:
                print(f"-- -- Error: {e}")
                correct = False
        print(f"THIS is hte result from the checking {correct}")
        return correct
    
    def process_facts(self, facts):
        
        print(f"-- -- GENERATED Facts: {facts}")
        #import pdb; pdb.set_trace()
        parsed_facts = []
        try:
            parsed_facts = ast.literal_eval(facts)
        except Exception as e:
            print(f"-- -- Error: {e}")
            try:
                pattern = r'\d+\.\s'
                parsed_facts = re.split(pattern, facts)
                parsed_facts = [s.strip() for s in parsed_facts if s]
            except Exception as e:
                print(f"-- -- Error: {e}")
        return parsed_facts
        
    def forward(self, text):
        #import pdb; pdb.set_trace()
        facts = self.generate_facts(text=text).claims

        print(f"-- -- Facts: {facts}")
        questions = []
        parsed_facts = self.process_facts(facts)
        
        if len(parsed_facts) == 0:
            print(f"-- -- No facts found")
            return questions, parsed_facts
        
        questions = [self.generate_question(fact=fact).question for fact in parsed_facts]
        print(f"-- -- Questions: {questions}")
        return questions, parsed_facts

qa_generator = QAGenerator()

#qa_generator = assert_transform_module(QAGenerator(), backtrack_handler)

In [3]:
doc = df_tpc.iloc[1].text
doc

'Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.'

In [217]:
questions, facts = qa_generator(text=doc)

-- -- Facts: 1. A small study in England in the early 1990s found an "association" between the vitamin K shot and childhood cancer.
2. An association means that two things are happening at the same time in the same person, but doesn't necessarily mean that one causes the other.
3. Scientists looked for the same association in other groups of children but never found it again in any other study.
4. There is no evidence to support the claim that the vitamin K shot causes cancer.
-- -- GENERATED Facts: 1. A small study in England in the early 1990s found an "association" between the vitamin K shot and childhood cancer.
2. An association means that two things are happening at the same time in the same person, but doesn't necessarily mean that one causes the other.
3. Scientists looked for the same association in other groups of children but never found it again in any other study.
4. There is no evidence to support the claim that the vitamin K shot causes cancer.
-- -- Error: invalid decim

In [219]:
facts

['A small study in England in the early 1990s found an "association" between the vitamin K shot and childhood cancer.',
 "An association means that two things are happening at the same time in the same person, but doesn't necessarily mean that one causes the other.",
 'Scientists looked for the same association in other groups of children but never found it again in any other study.',
 'There is no evidence to support the claim that the vitamin K shot causes cancer.']

In [52]:
class GenerateFacts(dspy.Signature):
    """Divide the text in atomic facts. Facts should have the form 'subject + verb + object'"""
    text = dspy.InputField()
    claims = dspy.OutputField(desc="List of atomic facts")

class FactsGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.ChainOfThought(GenerateFacts)

    def forward(self, text):
        facts = self.generate_facts(text=text).claims

        #print(f"-- -- Facts: {facts}")
        #parsed_facts = self.process_facts(facts)
    
        return facts

In [63]:
class GenerateFacts(dspy.Signature):
    """Extract the claims from the given text."""
    text = dspy.InputField(desc="may contain one or several claims")
    claims = dspy.OutputField(desc="List of claims in the form 'subject + verb + object'")

class FactsGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.ChainOfThought(GenerateFacts)

    def forward(self, text):
        facts = self.generate_facts(text=text).claims

        #print(f"-- -- Facts: {facts}")
        #parsed_facts = self.process_facts(facts)
    
        return facts

In [64]:
fact_generator = FactsGenerator()
fact_generator(doc)

'1. Urine tests can be used to detect specific substances.\n2. Certain substances in the urine may indicate a certain type of microcephaly.'

In [69]:
# llama
fact_generator = FactsGenerator()
fact_generator(doc)

for doc in df_tpc.text.values.tolist()[0:10]:
    print(doc)
    print(fact_generator(doc))

Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.
chorioamnionitis is a condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge. chorioamnionitis is a condition during pregnancy
Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If v

In [67]:
# gpt
# TODO: Create training dataset for this
fact_generator = FactsGenerator()
fact_generator(doc)

for doc in df_tpc.text.values.tolist()[0:10]:
    print(doc)
    print(fact_generator(doc))

Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.
1. Chorioamnionitis can cause unexplained fever during pregnancy.
2. Chorioamnionitis can cause uterine tenderness.
3. Chorioamnionitis can result in a high white blood cell count.
4. Chorioamnionitis can lead to a rapid heart rate in the fetus.
5. Chorioamnionitis can cause a rapid heart rate in the woman.
6. Chorioamnionitis can result in foul-smelling vaginal discharge.
Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood ca

In [60]:
llm.inspect_history(1)





Extract the claims from the given text.

---

Follow the following format.

Text: may contain one or several claims
Reasoning: Let's think step by step in order to ${produce the claims}. We ...
Claims: List of sentences in the form 'subject + verb + object' separated by ';'

---

Text: Urine test. This is done to look for a substance that may show a certain type of microcephaly.
Reasoning: Let's think step by step in order to produce the claims. We know that a urine test is done to look for a specific substance, which may indicate a certain type of microcephaly.
Claims: Urine test looks for a substance; Substance may indicate a certain type of microcephaly.





In [250]:
import pickle; results = pickle.load(open("example_doc0_10.pickle", "rb"))
results[0]

{'question': 'Did a small study in England in the early 1990s find an "association" between the vitamin K shot and childhood cancer?',
 'answer1': 'A small study in England in the early 1990s found an "association" between the vitamin K shot and childhood cancer.',
 'answer2': Prediction(
     rationale='produce the answer. We need to consider the information provided in the context, which clearly states that a small study in the early 1990s suggested a link between the vitamin K shot and cancer. However, larger studies conducted since then have found no connection between vitamin K and cancer. Therefore, based on the evidence presented, we can determine whether the small study in England did find an "association" between the vitamin K shot and childhood cancer.',
     answer='Yes, a small study in England in the early 1990s did find an "association" between the vitamin K shot and childhood cancer, but larger studies conducted since then have not found any connection between vitamin K 

In [225]:
results[0]["answer2"].context

['Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.',
 "You may have seen or heard comments from parents who don't want to get the shot for their baby. This is because there's still old information going around from a small study in the early 1990s. That study suggested a link between the

In [251]:
class CheckAnswersFaithfulness(dspy.Signature):
    """Verify that answers to the question are not contradictory."""
    question = dspy.InputField()
    answer1 = dspy.InputField()
    answer2 = dspy.InputField()
    faithfulness = dspy.OutputField(desc="boolean indicating if answer2 is faithful to answer1")
    rationale = dspy.OutputField()
    
class QAChecker(dspy.Module):
    def __init__(self):
        super().__init__()
        self.checker = dspy.ChainOfThought(CheckAnswersFaithfulness)#"text -> claims"
    
    def forward(self, answer1, answer2, question):
        response = self.checker(question=question, answer1=answer1, answer2=answer2)
        #print(f"-- -- faithfulness: {faithfulness}")
       
        return response.faithfulness, response.rationale
    
checker = QAChecker()

In [233]:
results[0].keys()

dict_keys(['question', 'answer1', 'answer2', 'text_passage'])

In [252]:
results_faith = []
for el in range(len(results[10:50])):
    faith, rationale = checker(results[el]["answer1"], results[el]["answer2"].answer, results[el]["question"])
    results_faith.append({
        "question": results[el]["question"],
        "answer1": results[el]["answer1"],
        "answer2": results[el]["answer2"].answer,
        "faith": faith,
        "rationale": rationale,
        "context": results[el]["answer2"].context,
        "text_passage": results[el]["answer2"].text_passage
        
    })
results_faith_df = pd.DataFrame(results_faith)

In [253]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       'display.max_colwidth',None
                       ):
    display(results_faith_df)

,question,answer1,answer2,faith,rationale,context,text_passage
0,"Did a small study in England in the early 1990s find an ""association"" between the vitamin K shot and childhood cancer?","A small study in England in the early 1990s found an ""association"" between the vitamin K shot and childhood cancer.","Yes, a small study in England in the early 1990s did find an ""association"" between the vitamin K shot and childhood cancer, but larger studies conducted since then have not found any connection between vitamin K and cancer.",true,"Answer 2 acknowledges the initial finding of an association between the vitamin K shot and childhood cancer in the small study in England in the early 1990s, but also states that larger studies conducted since then have not found any connection between vitamin K and cancer, which does not contradict the initial finding.","[You may have seen or heard comments from parents who don't want to get the shot for their baby. This is because there's still old information going around from a small study in the early 1990s. That study suggested a link between the vitamin K shot and cancer. Larger studies since then have found no connection between vitamin K and cancer. But the outdated information can still be found online., Is Vitamin K safe: A study from the early 1990’s found a possible link between getting vitamin K and developing childhood cancer. Pediatricians became very concerned about this and have done many studies since then, in many different ways, trying to see if this link was true. None of the studies found this link again, even though doctors and scientists looked very hard for it.]","Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study."
1,Does an association between two things happening at the same time in the same person necessarily mean that one causes the other?,"An association means that two things are happening at the same time in the same person, but doesn't necessarily mean that one causes the other.","No, an association between two things happening at the same time in the same person does not necessarily mean that one causes the other.",true,Both answers convey the same message that an association between two things happening at the same time in the same person does not imply causation.,"[In this five-year study, The National CADDRE Study: Child Development and Autism, a number of factors will be studied for their potential association with ASDs, including:\n- infections or abnormal responses to infections in the child, mother or father\n- genetic factors in the child, mother and father\n- mother′s reproductive history\n- abnormal hormone function in the child, mother or father\n- gastrointestinal problems in the child\n- family history of medical and developmental problems\n- smoking, alcohol and drug use in pregnancy, and\n- parentâ€™s occupation and other socio-demographic factors.., A woman's sexual habits and patterns can increase her risk for developing cervical cancer. Risky sexual practices include:\n- Having sex at an early age\n- Having multiple sexual partners\n- Having a partner or many partners who take part in high-risk sexual activities., Babies might have a higher risk of SIDS if:\n- their mother smoked, drank, or used drugs during pregnancy and after birth\n- their mothe

In [ ]:
# Define device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load model and tokenizer
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.to(device)

In [259]:
predictions = []
for row_id, row in results_faith_df.iterrows():
    premise = row["text_passage"]
    hypothesis = row["context"][0]
    inputs = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model(inputs["input_ids"])
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = ["entailment", "neutral", "contradiction"]
    #prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
    prediction = label_names[np.argmax(prediction)]
    predictions.append(prediction)

In [261]:
results_faith_df["nli_preds"] = predictions

In [263]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       'display.max_colwidth',None
                       ):
    display(results_faith_df[results_faith_df.nli_preds == "contradiction"])

,question,answer1,answer2,faith,rationale,context,text_passage,nli_preds
11,Will the daughter likely be a healthy carrier like her mother?,"If a daughter inherits the faulty gene, she will likely be a healthy carrier like her mother.",It is not guaranteed that the daughter will be a healthy carrier like her mother. Genetic testing and counseling are important to assess the risks and implications for the daughter's health.,false,"Answer 1 states that the daughter will likely be a healthy carrier like her mother, while answer 2 acknowledges that it is not guaranteed and that genetic testing and counseling are important to assess the risks. This shows a contradiction in the certainty of the outcome.","[Females pass one of their X chromosomes on to their child of either assigned sex. Therefore, they have a 50% chance of passing XLAS to any of their children., If a genetic test is ordered, the pediatrician or other healthcare professional will take a blood sample for testing. These tests can include:\n- Karyotype or chromosome microarray: This is used to assess the person for the presence of an extra X chromosome and to determine the degree of mosaicism (if any).\n- Prenatal genetic testing: Pregnant women who are at higher risk for having a child with triple X syndrome — mothers of advanced maternal age or women with triple X syndrome themselves — may be advised to undergo prenatal genetic testing, such as non-invasive prenatal testing (NIPT), amniocentesis or chorionic villi sampling (CVS)..]","Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.",contradiction
23,What are her clinical interests in the field of endocrinology?,"Her clinical interests include reproductive endocrine disorders such as premature ovarian insufficiency, polycystic ovary syndrome, precocious puberty, endometriosis, and complex contraception.","Dr. Camille Powe's clinical interests in the field of endocrinology include studying hormonal changes during pregnancy and their effects on maternal health, as well as applying this knowledge to improve outcomes for pregnant women.",false,"Answer 1 lists specific clinical interests in the field of endocrinology related to reproductive endocrine disorders, while answer 2 focuses on hormonal changes during pregnancy and maternal health. These two answers are not contradictory but rather provide different aspects of Dr. Camille Powe's clinical interests in endocrinology.","[A 34-year-old woman was referred for evaluation of ovarian enlargement and a pituitary mass. She had normal childhood development and regular menses until age 33, when her cycles became more frequent. She and her husband had been trying to conceive for approximately five years, but were unsuccessful., Endocrinologist Dr. Camille Powe, who co-leads the Diabetes in Pregnancy Program at Mass General, envisions a better future for these patients. She has devoted her career to better understanding how a growing baby affects the mother’s biology, and she is committed to applying that knowledge to help pregnant women benefit from these medical advances.]","Jacqueline Maher, M.D., is a reproductive endocrinology and infertility specialist with a focus on pediatric and adolescent gynecology. She leads the female fertility preservation program at Children's National Hospital to counsel patients in egg freezing or ovarian tissue freezing, and post treatment premature ovarian insufficiency, and hormone replacement/ puberty induction. Her other clinical interests include reproductive endocrine disorders such as premature ovarian insufficiency, polycystic ovary syndrome, precocious puberty, endometriosis and complex contraception; as well as surgery for congenital anomalies of the repro

## Classify questions

In [9]:
df_en = pd.read_csv("questions_rosie/FullTrialQa7152024.csv")
df_filtered = df_en.dropna(subset=['question', 'answerPassageText'])
df_filtered[['question', 'answerPassageText']]

,question,answerPassageText
0,How long should your kid nurse on each side?,How Long Does Nursing Take: Newborns may nurse...
1,What do I need to do if I have to be rescreene...,For people who do not receive a diagnosis of d...
2,How many ounces of milk should my 2 week old d...,How much milk should I store to meet my baby’s...
3,Do breastfed or formula babies spit up more?,Spitting up and dribbling milk with burps or a...
4,How do you get a breast pump from your insurance,How can I get a breast pump: Under the Patient...
...,...,...
16104,Thanks,Suggested citation for this article: Lengerich...
16105,What if I have a low lying placenta during pre...,What is the difference between placental abrup...
16106,Can you recommend any pelvic floor therapists ...,Do physical therapy: Your health care professi...
16107,Can I give birth vaginally with a placenta pre...,Can I still have a vaginal delivery with place...


In [14]:
len(df_filtered[['question', 'answerPassageText']].question.values.tolist())

16109

In [15]:
len(list(set(df_filtered[['question', 'answerPassageText']].question.values.tolist())))

11778

In [19]:
from sentence_transformers import SentenceTransformer
from sklearn.svm import OneClassSVM
import numpy as np

# Load pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose different models like 'paraphrase-MiniLM-L6-v2', etc.

def get_bert_embedding(text, tokenizer, model):
    # Tokenize input text and get embeddings
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the last hidden state (average of token embeddings)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()
# Sample positive questions (these are the ones from mothers)
positive_questions = list(set(df_filtered[['question', 'answerPassageText']].question.values.tolist()))

# Artificially generated questions (to be classified)
new_questions = [
    "Does the shot mentioned in the text temporarily decrease bone density?",
    "Can women improve their chances of having a baby born without a birth defect?",
]

# Generate embeddings for positive questions using SentenceTransformer
positive_embeddings = model.encode(positive_questions)

# Train One-Class SVM on the embeddings of positive questions
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
ocsvm.fit(positive_embeddings)

# Generate embeddings for new questions
new_embeddings = model.encode(new_questions)

# Predict on new questions
predictions = ocsvm.predict(new_embeddings)

# Output the predictions
for i, question in enumerate(new_questions):
    if predictions[i] == 1:
        print(f"'{question}' is classified as similar to the positive class (keep).")
    else:
        print(f"'{question}' is classified as not similar to the positive class (discard).")


/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/369 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Does the shot mentioned in the text temporarily decrease bone density?' is classified as not similar to the positive class (discard).
'Can women improve their chances of having a baby born without a birth defect?' is classified as similar to the positive class (keep).
